### Jolly Seber 

&nbsp;

Jolly-Seber model a statistical method to estimate the size of animal populations based upon mark-recapture data. The process is straight forward. In the first step, a sample of the population is captured and marked with chips or tags. After marking, the captured individuals are released back into the wild. After an interval of time, another sample is taken which may include both previously captured (marked) and new (unmarked) individuals. The model assumes equal capture probability between marked and unmarked so the ratio between marked and unmarked can derive the potential population size. However, the model assumes closed population without migration, birth or death. Many advanced models take into account of individual heterogeneity in capture and survival probabilities and time-varying capture and survival rates. These models can provide more accurate and precise population estimates while addressing the limitations of the basic Jolly-Seber model.

In [1]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
os.chdir('C:/Users/tm/Downloads/utas/thesis/chapter1/maria')

### cleanse

In [2]:
#read files
df=pd.ExcelFile('Maria_Island_fundamentals_2012_to_2019_NRETas.xlsx').parse('Sheet0')

In [3]:
#sort by date
grande=df[['INDIVIDUAL','OBSERVATION_DATE',]].sort_values(
    ['INDIVIDUAL','OBSERVATION_DATE'])
grande.reset_index(inplace=True,drop=True)

#datetimeindex
grande['OBSERVATION_DATE']=pd.to_datetime(grande['OBSERVATION_DATE'])

In [4]:
#for each year,each devil only counts once
grande['year']=grande['OBSERVATION_DATE'].dt.year
pop_raw=grande.loc[grande[['year','INDIVIDUAL']].drop_duplicates().index]

In [5]:
#sort by year
pop_raw=pop_raw.sort_values(['year','INDIVIDUAL'])

In [6]:
#create a dictionary which host last capture and recent capture date
dic={}
for i in pop_raw['year'].unique():
    for j in pop_raw['year'].unique():
        if i<j:
            dic[(i,j)]=0

In [7]:
#find last capture and recent capture date for each individual
for i in pop_raw['INDIVIDUAL'].unique():
    subset=pop_raw[pop_raw['INDIVIDUAL']==i].copy()
    if len(subset)>=2:
        dic[tuple(subset['year'].iloc[:2].tolist())]+=1

In [8]:
#compute recaptured animals in that year
total_marked=[sum([dic[j] for j in dic if j[1]==i]) for i in pop_raw['year'].unique()]

#compute total caught animals in that year
total_caught=pop_raw.groupby('year').count()['INDIVIDUAL'].tolist()

#compute the difference between caught and recaptured
total_unmarked=[total_caught[i]-total_marked[i] for i in range(len(total_caught))]

#always release animals caught
total_released=total_caught

In [9]:
total_caught_after=[]
total_recapture=[]
for i in pop_raw['year'].unique():
    year_before=[j for j in pop_raw['year'].unique() if j<i]
    year_after=[j for j in pop_raw['year'].unique() if j>i]
    
    #compute the animals caught before that year and recaptured after that year
    total_caught_after.append(sum([dic[(j,k)] for j in year_before for k in year_after]))
    
    #compute the animals caught that year and recaptured after that year
    total_recapture.append(sum([dic[(i,k)] for k in year_after]))

In [10]:
#estimate the population
pop_est=[]
for i in range(1,len(total_caught_after)-1):
    num=(total_released[i]*(total_caught_after[i])/(total_recapture[i])+total_marked[i])/((total_marked[i])/total_released[i])
    pop_est.append(num)

In [11]:
pop_est

[32.0, 49.0, 105.85227272727273, 170.94137931034481, 112.0, 81.99999999999999]